In [94]:
import numpy as np
import cv2
import os
import glob
import tensorflow
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, RandomFlip, RandomCrop, GlobalAveragePooling2D,BatchNormalization, LeakyReLU
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from dotenv import load_dotenv




## Data Preprocessing


In [121]:
class PreProcessing:
  """
  Goes through all images, returns preprocessed tensor.
  """

  def __init__(self, base_path):
    self.base_path = base_path
    self.kernel = np.array([[-1, -1, -1],
                  [-1, 8,-1],
                  [-1, -1, -1]])
  
  def get_colour_type(self, img_path):
    image = cv2.imread(img_path)
    if len(image.shape) == 3: return 3
    else: return 1

  def preprocess_image(self, img_path, height, width):
    """
    Function takes the path to the image and applys the preprocessing.
    """

    color_type = self.get_colour_type(img_path)

    if color_type == 1:
        img = cv2.imread(img_path, 0)
        img_gray = cv2.threshold(img,0,255,cv2.THRESH_TRUNC+cv2.THRESH_OTSU) 
        image_sharp = cv2.filter2D(src=img, ddepth=-1, kernel=self.kernel)

    elif color_type == 3:
        img = cv2.imread(img_path)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.threshold(img_gray,0,255,cv2.THRESH_TRUNC+cv2.THRESH_OTSU)
        image_sharp = cv2.filter2D(src=img, ddepth=-1, kernel=self.kernel)
        image_sharp = cv2.cvtColor(image_sharp, cv2.COLOR_BGR2GRAY)
    
    combined = cv2.add(image_sharp, img_gray[1])
    dst = cv2.resize(combined, (width, height))
    img = cv2.cvtColor(dst, cv2.COLOR_GRAY2BGR)
    print(type(img))
    return img
  
  

  def random_erasing(image, probability=0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
    """
    Function performs random erasing
    """
    # print(type(image))
    if np.random.uniform(0, 1) > probability:
        return image
    area = image.shape[0] * image.shape[1]
    for _ in range(100):
        target_area = np.random.uniform(sl, sh) * area
        aspect_ratio = np.random.uniform(r1, 1/r1)

        h = int(round(np.sqrt(target_area * aspect_ratio)))
        w = int(round(np.sqrt(target_area / aspect_ratio)))

        if w < image.shape[1] and h < image.shape[0]:
            x1 = np.random.randint(0, image.shape[0] - h)
            y1 = np.random.randint(0, image.shape[1] - w)
            if image.shape[2] == 3:
                image[x1:x1+h, y1:y1+w, 0] = mean[0]
                image[x1:x1+h, y1:y1+w, 1] = mean[1]
                image[x1:x1+h, y1:y1+w, 2] = mean[2]
            else:
                image[x1:x1+h, y1:y1+w, 0] = mean[0]
            return image
    return image

  def get_driver_data(self):
    """
    Returns a dictionary of image name as the key and driver and class as value.
    """
    driver_data = {}
    path = os.path.join(self.base_path,'driver_imgs_list.csv')

    print('Read drivers data')

    with open(path, 'r') as file:
      lines = file.readlines()
      lines = lines[1:]
    file.close()

    for line in lines:
      arr = line.strip().split(',')
      driver_data[arr[2]] = (arr[0], arr[1])
    
    return driver_data

  def load_data(self, height, width):
      """
      
      """
      x = []
      y = []
      driver_ids = []

      driver_data = self.get_driver_data()

      print('Read images')
      for class_number in range(10):
        print(f'Load folder c{class_number}')
        class_number_str = 'c' + str(class_number)
        path = os.path.join(self.base_path, 'imgs/data', class_number_str, '*.jpg')
        file_paths = glob.glob(path)  # Gets all file names matching given path.
        sub_x = []
        sub_y = []
        for file_path in file_paths:
            file_name = os.path.basename(file_path)
            image = self.preprocess_image(file_path, height, width)
            sub_x.append(image)
            temp = np.zeros(10)
            temp[class_number] = 1
            sub_y.append(temp)
            driver_id = driver_data[file_name][0]
            driver_ids.append(driver_id)
        x.append(sub_x)
        y.append(sub_y)
      return x, y, driver_ids

## Custom Data Augmentation Class


In [96]:
class RandomErasing(layers.Layer):
    """
    Class that performs Random Erasing in Random Erasing Data Augmentation by Zhong et al. 
    -------------------------------------------------------------------------------------
    probability: The probability that the operation will be performed.
    sl: min erasing area
    sh: max erasing area
    r1: min aspect ratio
    mean: erasing value
    ------
    Source: https://github.com/zhunzhong07/Random-Erasing/blob/master/transforms.py
    """
    def __init__(self, probability, sl, sh, r1, mean, **kwargs):
        self.probability = probability
        self.sl = sl
        self.sh = sh
        self.r1 = r1
        self.mean = mean
        super().__init__(**kwargs)        
    
    def __call__(self, image):
        return random_erasing_image(image, self.probability, self.sl, self.sh, self.r1, self.mean)


def random_erasing_image(image, probability=0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
    """
    Function performs random erasing
    """
    if np.random.uniform(0, 1) > probability:
        return image
    area = image.shape[0] * image.shape[1]
    for _ in range(100):
        target_area = np.random.uniform(sl, sh) * area
        aspect_ratio = np.random.uniform(r1, 1/r1)

        h = int(round(np.sqrt(target_area * aspect_ratio)))
        w = int(round(np.sqrt(target_area / aspect_ratio)))

        if w < image.shape[1] and h < image.shape[0]:
            x1 = np.random.randint(0, image.shape[0] - h)
            y1 = np.random.randint(0, image.shape[1] - w)
            if image.shape[2] == 3:
                image[x1:x1+h, y1:y1+w, 0] = mean[0]
                image[x1:x1+h, y1:y1+w, 1] = mean[1]
                image[x1:x1+h, y1:y1+w, 2] = mean[2]
            else:
                image[x1:x1+h, y1:y1+w, 0] = mean[0]
            return image
    return image
        

def random_erasing(image, probability = 0.5, sl = 0.02, sh = 0.4, r1 = 0.3, mean=[0.4914, 0.4822, 0.4465]):
    """
    lamdba layer.
    """
    return layers.Lambda(lambda: random_erasing_image(image, probability, sl, sh, r1, mean))


## Splitting the data.

In [120]:
def split_data(x, y):
  x_train = []
  y_train = []
  x_test = []
  y_test = []
  
  split_points = percent_indexes(x)
  for class_num, (xi, yi) in enumerate(zip(x, y)):
    
    for image_number, (image, out) in enumerate(zip(xi, yi)):
      if image_number < split_points[class_num]:
        x_test.append(image)
        y_test.append(out)
        
      else:
        image = random_erasing(image)
        print(type(image))
        x_train.append(image)
        y_train.append(out)
  return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)
  

def percent_indexes(x):
  split_points = []
  for xi in x:
    number_of_images = len(xi)
    split_point = int(number_of_images*0.2)
    split_points.append(split_point)
  return split_points

## VGG16 Model

In [114]:
class VGG16:

  def __init__(self):
    self.vgg16 = self.build_model()
    self.vgg16.summary()

  def build_model(self, input_shape=(None,None,3)):
    model = Sequential([
      Input(shape=input_shape),                               # Block 1
      Conv2D(32, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(32, (3, 3), padding='same'),
      BatchNormalization(axis = 3),
      LeakyReLU(),
      MaxPool2D((2, 2), strides=(2, 2)),                      # Block 2
      Conv2D(64, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(64, (3, 3), padding='same'),
      BatchNormalization(axis = 3),
      LeakyReLU(),
      MaxPool2D((2, 2), strides=(2, 2)),                      # Block 3
      Conv2D(128, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(128, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(128, (3, 3), padding='same'),
      BatchNormalization(axis = 3),
      LeakyReLU(),
      MaxPool2D((2, 2), strides=(2, 2)),                      # Block 4  
      Conv2D(256, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(256, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(256, (3, 3), padding='same'),
      BatchNormalization(axis = 3),
      LeakyReLU(),
      MaxPool2D((2, 2), strides=(2, 2)),                      # Block 5  
      Conv2D(256, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(256, (3, 3), padding='same'),
      BatchNormalization(),
      LeakyReLU(),
      MaxPool2D((1, 1), strides=(1, 1)),
      Conv2D(256, (3, 3), padding='same'),
      BatchNormalization(axis = 3),
      LeakyReLU(),
      MaxPool2D((2, 2), strides=(2, 2)),                      # Fully Connected Layers
      GlobalAveragePooling2D(),
      Dense(512, activation='relu'),
      Dense(10, activation='softmax')
    ])
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
  
  def train_model(self, x_train, y_train, x_test, y_test):
    self.vgg16.fit(x_train, y_train, epochs=10, verbose=1, batch_size=16)
    test_loss, test_acc = self.vgg16.evaluate(x_test, y_test)
    print(f'\nTest lost: {test_loss} -- Test accuracy: {test_acc}')

## Load Data from file.

In [122]:
HEIGHT = 64
WIDTH = 64

load_dotenv()
PATH = os.getenv('PATH_TO_DATA')
p = PreProcessing(PATH)
x, y, driver_ids = p.load_data(HEIGHT, WIDTH)

x_train, y_train, x_test, y_test = split_data(x, y)

Read drivers data
Read images
Load folder c0
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarr

KeyboardInterrupt: 

## Run the model.

In [111]:
model = VGG16()

model.train_model(x_train, y_train, x_test, y_test)

Model: "sequential_78"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_521 (Conv2D)         (None, None, None, 32)    896       
                                                                 
 batch_normalization_521 (Ba  (None, None, None, 32)   128       
 tchNormalization)                                               
                                                                 
 leaky_re_lu_213 (LeakyReLU)  (None, None, None, 32)   0         
                                                                 
 max_pooling2d_312 (MaxPooli  (None, None, None, 32)   0         
 ng2D)                                                           
                                                                 
 conv2d_522 (Conv2D)         (None, None, None, 32)    9248      
                                                                 
 batch_normalization_522 (Ba  (None, None, None, 32) 

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Lambda).